In [1]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Citeseer')
data = cora_dataset[0].to(device)
print(data)
trials = 5

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [4]:
edges_to_add = None

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model_save_path = "../models/citeseer_gcn_model.pth"
list_save_path = "../attacks/citeseer_gcn_edges_full_run.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [7]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.69
Initial Loss: 1.2167097330093384


In [9]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [ ]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_mcmc(data, train, model, 0.0, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.9855559468269348
Surrogate acc on unlabled data: 0.6990000000000001
Attack loss: 0.32994788885116577
Surrogate loss on unlabled data: 1.2558293342590332
Surrogate acc on unlabled data: 0.642
Attack loss: 0.6012729406356812
Surrogate loss on unlabled data: 1.3341989517211914
Surrogate acc on unlabled data: 0.628
Attack loss: 0.7639239430427551
Surrogate loss on unlabled data: 1.420682668685913
Surrogate acc on unlabled data: 0.613
Attack loss: 0.9208765029907227
Surrogate loss on unlabled data: 1.6383326053619385
Surrogate acc on unlabled data: 0.562
Attack loss: 1.133531093597412
Surrogate loss on unlabled data: 1.6173580884933472
Surrogate acc on unlabled data: 0.554
Attack loss: 1.184903860092163
Surrogate loss on unlabled data: 1.7345726490020752
Surrogate acc on unlabled data: 0.549
Attack loss: 1.2981828451156616
Surrogate loss on unlabled data: 2.0600335597991943
Surrogate acc on unlabled data: 0.524
Attack loss: 1.6107633113861084
Surrogate los

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.9931710958480835
Surrogate acc on unlabled data: 0.687
Attack loss: 0.3328549861907959
Surrogate loss on unlabled data: 1.1444406509399414
Surrogate acc on unlabled data: 0.67
Attack loss: 0.5647733211517334
Surrogate loss on unlabled data: 1.3164896965026855
Surrogate acc on unlabled data: 0.631
Attack loss: 0.7717063426971436
Surrogate loss on unlabled data: 1.5090131759643555
Surrogate acc on unlabled data: 0.61
Attack loss: 0.9843701720237732
Surrogate loss on unlabled data: 1.5806121826171875
Surrogate acc on unlabled data: 0.578
Attack loss: 1.0521540641784668
Surrogate loss on unlabled data: 1.7531664371490479
Surrogate acc on unlabled data: 0.545
Attack loss: 1.2412980794906616
Surrogate loss on unlabled data: 1.825074315071106
Surrogate acc on unlabled data: 0.529
Attack loss: 1.3524757623672485
Surrogate loss on unlabled data: 1.9374980926513672
Surrogate acc on unlabled data: 0.521
Attack loss: 1.4676785469055176


# regular, secondary attack, not reversed

In [ ]:
splits = [0.0, 0.5]

split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_loss, split_dic_acc, itrs_one, Model.GCN, Dataset.CITESEER, AcceptFn.CONSTANT, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"citeseer"}_{"constant"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_acc, ptb_rate, "Greedy", "", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_loss, ptb_rate, "Greedy", "", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [ ]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_decreasing_loss, split_dic_decreasing_acc, itrs_two, Model.GCN, Dataset.CITESEER, AcceptFn.INCREASING, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"citeseer"}_{"increasing"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_acc, ptb_rate, "Greedy", "", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_loss, ptb_rate, "Greedy", "", "increasing", "loss", "regular")